In [ ]:
###########import packages##########
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.linear_model import BayesianRidge
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import ExtraTreeClassifier
from sklearn import linear_model
import lightgbm
import catboost
from catboost import *
import xgboost
import shap
%matplotlib
###########loading data##########
fdata=pd.read_csv('database_filled_CD.csv',encoding="gbk")
raw_data=fdata.loc[:,[
                       'Operating Temperature (℃)',#0
                      'Operating Pressure (bar)',#1
                      'Flow Rate (mL min-1)',#2
                      'Active Area (cm2)', #3
                      'Ir wt. %',#4
                      'Ru wt.%',#5
                      'O wt. %',#6
                      'C wt. %',#7
                      'Pure_0/Supported_1',#8
                      'I/C in Anode',#9
                      'Pt wt. %',#10
                      'I/C in Cathode',#11  
                      'Anode Precious Metal Loading (mg cm-2 Ir/Ru/Pt/Pd)',#12
                      'Cathode Precious Metal Loading (mg cm-2 Pt/Pd)',#13
                      'CCM_0/GDE_1',#14    
                      'Membrane Thickness (μm)',#15
                      'EW',#16
                      'Minimum Stability Current Density (A cm-2)',#17
                      'Maximum Stability Current Density (A cm-2)',#18
                      'Fluctuation period (h)',#19
                      'Stability Test Time (h-1)',#20
                      'CL_2V_2A'#21
                        ]]
###########train test splitting##########
raw_param=raw_data.iloc[:,0:21]
raw_power=raw_data.iloc[:,21]
print('ready')

In [ ]:
from sklearn.metrics import roc_curve, auc
def auc_curve(y_label, y_pre,algorithm_name):
    y_label = y_label + 1
    y_pre = y_pre + 1
    fpr, tpr, thersholds = roc_curve(y_label, y_pre, pos_label=2)
    roc_auc = auc(fpr, tpr)
    x_line=np.arange(0,1.01,0.01)
    y_line=np.arange(0,1.01,0.01)
    print('auc',roc_auc)
    plt.plot(fpr, tpr, 'k--', label='ROC (AUC/area = {0:.2f})'.format(roc_auc), lw=2)
    plt.plot(x_line,y_line,c='red')
    plt.xlim([-0.05, 1.05])  # 设置x、y轴的上下限，以免和边缘重合，更好的观察图像的整体
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')  # 可以使用中文，但需要导入一些库即字体
    plt.title('ROC Curve of %s' %algorithm_name)
    plt.legend(loc="lower right")
def gridsearch(model,param,algorithm_name):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5,n_jobs=-1)
    grid.fit(X_train,y_train)
    print('Best Classifier:',grid.best_params_,'Best Score:', grid.best_score_)
    best_model=grid.best_estimator_
    prediction_train=best_model.predict(X_train)
    prediction_test=best_model.predict(X_test)
    final_result=classification_report(y_test,prediction_test,output_dict=True)
    ##############################################################
    y_score=best_model.predict_proba(X_test)
    y_score=y_score[:,1]
    auc_curve(y_test,y_score,algorithm_name)
    ##############################################################
    print(classification_report(y_train,prediction_train))
    print(classification_report(y_test,prediction_test))
    print(final_result['accuracy'])
    ###########generating a figure##########
    print(algorithm_name)
    print(best_model.feature_importances_)
def shap_plot(model,param,algorithm_name):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:21]
    SHAP_OUTPUT=raw_data.iloc[:,21]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values[1], SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values[0]).mean(0)
        print(global_importances)
    else:
        explainer = shap.TreeExplainer(best_model,X_SHAP)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
#         shap.dependence_plot('Electronegativity',shap_values,SHAP_INPUT,interaction_index='Number of d electrons')
#         shap.dependence_plot('Number of d electrons',shap_values,SHAP_INPUT,interaction_index='Electronegativity')
#         shap.summary_plot(shap_values, SHAP_INPUT,max_display=100)
        global_importances = np.abs(shap_values).mean(0)
        print(global_importances)

In [ ]:
seed=63
X_train, X_test, y_train, y_test = train_test_split(raw_param, raw_power, test_size=.15,random_state=seed)
##########LGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt','rf'],
 'learning_rate':[0.5,0.6,0.7,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1,1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2,1.3,1.4,1.5],
  'n_estimators':[50,100,200,400,None],
 'subsample':[0.3,0.35,0.4,0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.5,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.6,0.65,0.7],
 'max_depth':[5,7,9,11,-1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbosity=0)
param_xg = {
 'booster':['gbtree'],
  'learning_rate':[0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],
 'n_estimators':[50,100,200,400,None],
 'max_depth':[5,7,9,11,16],
 'subsample':[0.6,0.65,0.7,0.8,0.85,0.9,0.95,1],
 'reg_alpha':[0,0.001,0.01,0.0001,0.00001],
 'reg_lambda':[0,0.001,0.01,0.0001,0.00001]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
     'learning_rate':[0.001,0.0025,0.005,0.0075,0.01,0.025,0.05,0.075,0.1,0.25,0.5,0.75,1],
 'n_estimators':[50,100,200,400],
"boosting_type":["Plain"],
 'max_depth':[5,7,9,11],
 'subsample':[0.4,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1],
    'reg_lambda':[0,0.001,0.01,0.0001,0.00001]

}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.2,0.25,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6,0.62,0.64,0.66,0.68,0.7],
     'max_depth':[3,5,7,9,11,16],
 'criterion':['friedman_mse','mae','mse'],
 'max_features':['auto','sqrt','log2'],
 'loss':['deviance', 'exponential']
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
  'n_estimators':[10,50,100,200,400],
     'max_depth':[3,5,7,9,11,None],
     'criterion':['gini','entropy'],
     'max_features':['auto','sqrt','log2']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')


###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
    'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')


###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
    'max_depth':[5,6,7,8,9,10,11,None],
        'criterion' : ['gini','entropy'],
        'splitter' : [ "best",'random'],
        'max_features':['auto','sqrt','log2']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')


###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
      'learning_rate':[0.5,0.6,0.7,0.82,0.84,0.86,0.88,0.9,0.92,0.94,0.96,0.98,1,1.02,1.04,1.06,1.08,1.1,1.12,1.14,1.16,1.18,1.2,1.3,1.4,1.5],
    'n_estimators':[30,40,50,100,200]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

In [ ]:
##########LightGBM gridsearch CV for best hyperparameter##########
model_LightGBMClassifier=lightgbm.LGBMClassifier(random_state=1,verbose=-1)
param_light = {
'boosting_type':['gbdt'],
 'learning_rate':[0.7],
  'n_estimators':[50],
 'subsample':[0.3],
 'max_depth':[7],
 'reg_alpha':[0],
 'reg_lambda':[0.001]
}
gridsearch(model_LightGBMClassifier,param_light,'LightGBM')

In [ ]:
shap_plot(model_LightGBMClassifier,param_light,'LightGBM')

In [ ]:
##########XGBoost gridsearch CV for best hyperparameter##########
model_XGBClassifier=xgboost.XGBClassifier(objective ='reg:squarederror',random_state=1,verbose=0)
param_xg = {
'booster':['gbtree'],
'learning_rate':[0.36],
'n_estimators':[50],
'max_depth':[5],
'subsample':[1],
'reg_alpha':[0.0001],
'reg_lambda':[0.001]
}
gridsearch(model_XGBClassifier,param_xg,'XGBoost')

In [ ]:
shap_plot(model_XGBClassifier,param_xg,'XGBoost')

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatClassifier=catboost.CatBoostClassifier(random_state=1,verbose=0)
param_cat = {
'learning_rate':[0.1],
'n_estimators':[50],
'max_depth':[5],
'subsample':[0.9],
'reg_lambda':[0.01]
}
gridsearch(model_CatClassifier,param_cat,'CatBoost')

In [ ]:
shap_plot(model_CatClassifier,param_cat,'CatBoost')

In [ ]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingClassifier = ensemble.GradientBoostingClassifier(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
'learning_rate':[0.3],
'criterion':['friedman_mse'],
'max_features':['sqrt'],
'loss':['deviance'],
'max_depth':[9]
}
gridsearch(model_GradientBoostingClassifier,param_GB,'GradientBoost')

In [ ]:
shap_plot(model_GradientBoostingClassifier,param_GB,'GradientBoost')

In [ ]:
###########RandomForest gridsearch CV for best hyperparameter##########
model_RandomForestClassifier = ensemble.RandomForestClassifier(random_state=1)
###########defining the parameters dictionary##########
param_RF = {
'n_estimators':[50],
'max_depth':[None],
'criterion':['gini'],
'max_features':['auto']
}
gridsearch(model_RandomForestClassifier,param_RF,'Random Forest')

In [ ]:
shap_plot(model_RandomForestClassifier,param_RF,'Random Forest')

In [ ]:
###########Extra Tree gridsearch CV for best hyperparameter##########
model_ExtraTreeClassifier = ExtraTreeClassifier(random_state=1)
param_ET = {
'max_depth':[None],
'criterion' : ['entropy'],
'splitter' : [ "random"],
'max_features':['auto']
}
gridsearch(model_ExtraTreeClassifier,param_ET,'Extra Tree')

In [ ]:
shap_plot(model_ExtraTreeClassifier,param_ET,'Extra Tree')

In [ ]:
###########Decision Tree gridsearch CV for best hyperparameter##########
model_DecisionTreeClassifier = tree.DecisionTreeClassifier(random_state=1)
param_DT = {
'max_depth':[None],
'criterion' : ['entropy'],
'splitter' : [ "random"],
'max_features':['auto']
}
gridsearch(model_DecisionTreeClassifier,param_DT,'Decision Tree')

In [ ]:
shap_plot(model_DecisionTreeClassifier,param_DT,'Decision Tree')

In [ ]:
###########AdaBoost gridsearch CV for best hyperparameter##########
model_AdaBoostClassifier = ensemble.AdaBoostClassifier(random_state=1)
param_Ada = {
'n_estimators':[100],
'learning_rate':[0.88]
}
gridsearch(model_AdaBoostClassifier,param_Ada,'AdaBoost')

In [ ]:
shap_plot(model_AdaBoostClassifier,param_Ada,'AdaBoost')